In [1]:
!pip install torch torchvision


In [2]:
import os
import shutil
import random
import numpy as np
from collections import defaultdict,Counter
from PIL import Image
from torchvision import transforms


In [3]:
# oversampling the training dataset 
images_dir = r"train/images"
labels_dir = r"train/labels"

output_dir = r"train_oversampled"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(f"{output_dir}/images", exist_ok=True)
os.makedirs(f"{output_dir}/labels", exist_ok=True)

In [4]:
# these are the test results obtained from the yolov12s model
results = {"per_class_metrics":[
{"Class": "Ants","mAP50":0.81353,"Box-R": 0.71264},
    {"Class":"Bees", "mAP50":0.9322, "Box-R": 0.86364},
    {"Class":"Beetles", "mAP50": 0.80151, "Box-R": 0.68182},
    {"Class": "Caterpillars","mAP50": 0.56236, "Box-R": 0.44584},
        {"Class": "Earthworms",  "mAP50": 0.50175, "Box-R": 0.40672},
        {"Class": "Earwigs","mAP50": 0.73984, "Box-R":0.61355},
        {"Class": "Grasshoppers", "mAP50": 0.61363, "Box-R":0.55109},
        {"Class":"Moths","mAP50":  0.98688, "Box-R":0.93617},
        {"Class":"Slugs",   "mAP50": 0.73489, "Box-R": 0.66667},
        {"Class":"Snails",  "mAP50":0.89511,"Box-R":0.88},
        {"Class":"Wasps", "mAP50": 0.96977,"Box-R":0.97872},
        {"Class":"Weevils","mAP50":0.99364, "Box-R":1.0}]}

cls_names = [m["Class"] for  m in results["per_class_metrics"]]
mAPs = [m["mAP50"] for m  in results["per_class_metrics"]]
recalls = [m["Box-R"] for  m in results["per_class_metrics"]]

"""below is the formula that I used for
computing the oversampling weights, we give
more importance to the mAP metric

"""
combined_score = np.array(mAPs)*0.6 + np.array(recalls)*0.4
inv_weights =1.0 /(combined_score +1e-10)
norm_weights = inv_weights / inv_weights.mean()  # normalizing thw weights

weights = dict(zip(cls_names, norm_weights))
print("Oversampling weights (based on how badly yolov12s perfomed on that class):")
for k, v in weights.items():
    print(f"{k} : {v:.2f}")

Oversampling weights (based on how badly yolov12s perfomed on that class):
Ants : 0.94
Bees : 0.80
Beetles : 0.96
Caterpillars : 1.40
Earthworms : 1.56
Earwigs : 1.05
Grasshoppers : 1.23
Moths : 0.75
Slugs : 1.02
Snails : 0.81
Wasps : 0.74
Weevils : 0.73


In [7]:
class_to_indices = defaultdict(list)
label_files = sorted(os.listdir(labels_dir))

for idx, lf in enumerate(label_files):

    with open(os.path.join(labels_dir,lf),"r") as f:
        lines = f.read().strip().splitlines()

    classes_in_img = {int(line.split()[0]) for  line in lines}
    
    for cls in classes_in_img:
        class_to_indices[cls].append(idx)

N = len(label_files)
print(f"Total original images: {N}")

Total original images: 11502


In [8]:
print("IMAGES PER CLASS in the Orignal  training dataset : ")
for cls_id,name in enumerate(cls_names):
    count =len(class_to_indices.get(cls_id,[]))
    print(f"{name:} : {count} images")


IMAGES PER CLASS in the Orignal  training dataset : 
Ants : 1032 images
Bees : 1101 images
Beetles : 857 images
Caterpillars : 912 images
Earthworms : 720 images
Earwigs : 942 images
Grasshoppers : 1044 images
Moths : 1059 images
Slugs : 797 images
Snails : 1085 images
Wasps : 1014 images
Weevils : 960 images


In [9]:
# the new training dataset is the orignal data + the oversampled data
print("Copying original training images and their abels")

for img_file in os.listdir(images_dir):
    shutil.copy(os.path.join(images_dir, img_file),
        os.path.join(output_dir,"images", img_file))

for lbl_file in os.listdir(labels_dir):
    shutil.copy(os.path.join(labels_dir,lbl_file),
        os.path.join(output_dir,  "labels",lbl_file))

print("Done")

Copying original training images and their abels
Done


In [10]:
os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)

label_files = sorted(os.listdir(labels_dir))
class_to_indices = defaultdict(list)

for idx, txt_file in enumerate(label_files):
    with open(os.path.join(labels_dir, txt_file)) as f:
        lines = f.read().strip().splitlines()

    classes = {int(line.split()[0]) for line in lines}

    for cls in classes:
        class_to_indices[cls].append(idx)

# cls to id number mapping
class_to_idx = {name: i for i,name in enumerate(cls_names)}


sampled_indices = []
max_oversampling = 2.0  # at max we double the amount of data although that wont happen in our case
extra_counts_per_class =  {name:0 for name in cls_names}

for cls_name,weight in weights.items():

    if weight <=1.0:
        continue  # only oversample weak classes

    cls_idx = class_to_idx[cls_name]
    orig_indices =class_to_indices.get(cls_idx, [])

    if not orig_indices:
        continue

    multiplier =  min(weight,max_oversampling)
    desired_count =  int(round(len(orig_indices) *multiplier))
    extra_needed =  desired_count - len(orig_indices)

    if extra_needed <= 0: # no extra need for class then skip
        continue

    sampled = random.choices(orig_indices,k=extra_needed)
    sampled_indices.extend(sampled)
    extra_counts_per_class[cls_name] = extra_needed

print("Oversampling summary:")
print("Original images:",len(label_files))

print("Extra total images oversampled:",len(sampled_indices))

for k, v in extra_counts_per_class.items():
    if v > 0:
        print(f"{k}: +{v}")


Oversampling summary:
Original images: 11502
Extra total images oversampled: 1077
Caterpillars: +368
Earthworms: +404
Earwigs: +47
Grasshoppers: +240
Slugs: +18


In [11]:
!pip install albumentations

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.4),
        A.Rotate(limit = 15, p = 0.5, border_mode=0)],   # rotation without using albumentations leads to incorrect bboxes
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'],
        min_visibility = 0.2))

print("Adding oversampled images with correct bboxes")

counter =0
for idx in sampled_indices:

    #loading images
    label_file = label_files[idx]
    img_file =label_file.replace(".txt",".jpg")

    img_path =os.path.join(images_dir,img_file)
    label_path =os.path.join(labels_dir, label_file)

    if not os.path.exists(img_path):
        img_file =img_file.replace(".jpg", ".png")
        img_path = os.path.join(images_dir, img_file)

    image = np.array(Image.open(img_path).convert("RGB"))

    bboxes =[]
    cls_ids =[]

    with open(label_path,"r") as f:
        for line in f.read().strip().split("\n"):

            parts =line.split()
            cls =int(parts[0])
            cx,cy, w,h = map(float,parts[1:])

            bboxes.append([cx,cy,w,h])
            cls_ids.append(cls)

    # using almbumentations so that the bboxes can also be changed according to the augmentations
    augmented = transform(image = image, bboxes=bboxes,class_labels=cls_ids)

    aug_img =augmented["image"]
    aug_bboxes =augmented["bboxes"]
    aug_classes =augmented["class_labels"]

    # saving the augmented imgs
    new_img_name =f"aug_{counter}_{img_file}"
    Image.fromarray(aug_img).save(os.path.join(output_dir, "images", new_img_name))

    # saving back in yolo format
    new_label_name = f"aug_{counter}_{label_file}"
    with open(os.path.join(output_dir, "labels", new_label_name), "w") as f:
        for cls, (cx,cy,w,h) in zip(aug_classes, aug_bboxes):
            f.write(f"{cls} {cx :.6f} {cy :.6f} {w:.6f}{h:.6f}\n")

    counter +=1

print("Albumentations oversampling done")


c:\Users\suraj\OneDrive\Desktop\COMP9517\Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Adding oversampled images with correct bboxes
Albumentations oversampling done


### Now we zip all of the splits that is oversampled training data, testing data and validation data into one zip file so that it can uploaded to colab and the model can be trained on it.